# A compact statistical model of Bengalese finch song syntax
## A case study in reproducibility
David Nicholson  
post-doctoral fellow    
Emory University  
nee Sober lab, now Prinz lab

## Introduction
### songbirds
- learn their song during *critical period*
- by hearing a *tutor* and practicing an imitation
<img src="./images/zeebies_sakata_picture_dooling_spectrogram.png" alt="tutoring" style="width: 300px;float: right;"/>

### the song system
- songbird brain very similar to human brain
<img src="./images/classic_modern_brain_evolution.gif" alt="bird_v_human_brain" style="width: 500px;float: center;"/>
(Image credit: Zina Deretsky - NSF)

- song system evolved "on top of" areas found in all brains

<img src="./images/evolution_vocal_pathways.jpg" alt="bird_v_human_brain" style="width: 600px;float: center;"/>

Chakraborty, Mukta, and Erich D. Jarvis. "Brain evolution by brain pathway duplication." Phil. Trans. R. Soc. B 370.1684 (2015): 20150056.

- **therefore songbirds provide a *model system* to understand  
  how the brain learns and produces  
  sequenced, precisely-controlled behaviors learned by imitation and practice, like:**  
  - speaking a language
  - playing the guitar
  - typing a computer program

###  Song
<img src="./images/example_song.png" alt="example_song" style="width: 600px;float: right;"/>
Song *is* a sequenced and precisely controlled behavior
- Most species have song made up of elements called **syllables**
- The bird sings **sequences** of these syllables
- The sequences obey a set of rules, i.e., a **syntax**, similar to the way that speech obeys a syntax
- If you want to know how the brain produces a behavior,  
  you need to be able to quantitatively describe that behavior
  such as a **generative model**
- if my **model of the brain** produces behavior indistinguishable from my **model of behavior**, then
  I could argue I understand something about how the brain works

### Different species, different songs with different syntax
#### Zebra finch

- commonly studied by neuroscientists (easy to bred in lab)
- squawky song
- simple syntax
  - well described by a **first-order Markov model** where the probability of a given syllable occuring at a given "time step" in a sequence depends only on syllable that occurred at previous time step

<img src="./images/Hessler_Doupe_1999_Fig1.png" alt="hessler_doupe_fig1" style="width: 500px;float: center;"/>
Figure 1 from N.A. Hessler and A.J. Doupe, "Singing-Related Neural Activity in a Dorsal Forebrain–Basal Ganglia Circuit of Adult Zebra Finches", J. Neuroscience 1999

#### Bengalese finch

- can contain **branch points** where one syllable can transition to two or more other syllables
- can also include **repeats** where a syllable transitions back to itself
<img src="./images/Wohlgemuth_Sober_Brainard.jpg" alt="wohlgemuth_sober_brainard_2010" style="width: 500px;float: center;"/>
Figure 1 from Wohlgemuth, Melville J., Samuel J. Sober, and Michael S. Brainard. "Linked control of syllable sequence and phonology in birdsong." Journal of Neuroscience 30.39 (2010): 12936-12949.

### Bengalese finch song can be described by a Partially-Observable Markov Model with Adaptation ("POMMA")

<img src="./images/journal.pcbi.1001108.g007.png" alt="Jin_Kozhevnikov_fig7" style="width: 500px;float: center;"/>

Jin, Dezhe Z., and Alexay A. Kozhevnikov. "A compact statistical model of the song syntax in Bengalese finch." PLoS computational biology 7.3 (2011): e1001108.

I am working with one of the authors, Dezhe Jin, to convert code for fitting POMMAs to a Python package:
https://github.com/NickleDave/pomma

### Bengalese finch song syntax does not obey first order Markov model
#### in particular, repeat distributions cannot be reproduced by a first-order Markov process

I show this by fitting syntax of one bird's sequences with a first-order Markov model.

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import pomma

First, I load some data.

In [ ]:
data = pomma.datasets.load()
labels = data['bl26lb16']  # bird ID

The sequences of labels looks like this,  
where each row is a sequence from one song:

In [ ]:
for ind in range(5):
    print(labels[ind])

I make a first-order Markov transition matrix based on the sequences of syllable labels.

In [ ]:
startchar = 'S'
endchar = 'E'
labelset = list(startchar + 'iabcdefg' + endchar)
tr = pomma.first_order_markov.make_trans_mat(labels, labelset, endchar)

This is what such a matrix looks like for this data:

In [ ]:
pomma.plot_utils.plot_trans_mat(tr, labelset)

I then use the transition matrix to generate many sequences.

In [ ]:
seqs = pomma.first_order_markov.generate_sequences(tr, labelset, num=500)
for seq in seqs[:5]:
    print(seq)

Now let's look at the distribution of repeats in the actual data and in the generated sequences.

In [ ]:
# from the actual data
labels_repeat_distribs = pomma.statistics.get_repeat_distribs(labels, 'iabcdefghjk')
counts_b = labels_repeat_distribs['b']['counts']
unique_repeats_b = labels_repeat_distribs['b']['unique_repeats']
n_b = [len(uniq)-1 for uniq in unique_repeats_b]
p_counts = np.asarray(counts_b) / sum(counts_b)
p_init = tr[2,3]

# from the generated sequences
seqs_repeat_distribs = pomma.statistics.get_repeat_distribs(seqs, 'iabcdefghjk')
counts_b_seqs = seqs_repeat_distribs['b']['counts']
unique_repeats_b_seqs = seqs_repeat_distribs['b']['unique_repeats']
n_b_seqs = [len(uniq)-1 for uniq in unique_repeats_b_seqs]
p_counts_b_seqs = np.asarray(counts_b_seqs) / sum(counts_b_seqs)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(8,6)
ax.plot(n_b, p_counts, color='blue', label='data')
ax.plot(n_b_seqs[:len(n_b)], p_counts_b_seqs[:len(n_b)], color='red', linestyle='--', label='Markov model')
ax.set_title('probability distribution of number of repeats')
ax.set_ylabel('probability')
ax.set_xlabel('number of repeats')
ax.set_xticks(n_b);
ax.legend();

** Notice that the repeat distribution from the model (red dashed line) peaks at the start and then decays exponentially, while the peak repeat distribution in the real data has a peak at 7, not at the start**
## To better fit these repeat distributions, we introduce the idea of *adaptation*

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def prob_n_repeats_adapt(p_init, n, alpha):
    """probability of seeing n repeats under adaptation
    p_init : float
        initial probability of the syllable transitioning back to itself
    n : int
        number of repeats of syllable
    alpha : float between 0 and 1
    """
    
    if alpha < 0 or alpha > 1:
        raise ValueError('value of alpha must be bewteen 0 and 1')
    
    return (alpha ** (
        (n - 2) * (n - 1) / 2)
           ) * (p_init ** (n-1)) * (
        1 - (alpha ** (n-1) * p_init)
    )

In [ ]:
def p_curve(repeat_nums, p_init, alpha):
    return np.asarray(
        [prob_n_repeats_adapt(p_init, n=n, alpha=alpha)
         for n in repeat_nums]
    )

In [ ]:
p_init = tr[2,3]
p_opt, p_cov = curve_fit(lambda x, alpha: p_curve(x, p_init, alpha),
                         xdata=n_b,ydata=p_counts,
                         p0=0.75,
                         bounds=((0.5,),(1.0,)))

In [ ]:
y_hat = p_curve(n_b,p_init,p_opt)
x = n_b
y = p_counts
fig, ax = plt.subplots()
ax.plot(x, y)
ax.plot(x, y_hat, linestyle='--');

## To be continued ...